### Producer

In [1]:
import pandas as pd
import json 
from time import sleep

In [ ]:
from json import dumps
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers=['YOUR PUBLIC IP:9092'], 
                          value_serializer=lambda x:
                           dumps(x).encode('utf-8'))

In [ ]:
producer.send('demo_testing2', value="{'username':'wawex'}")

In [ ]:
df = pd.read_csv("data/indexProcessed.csv")

In [ ]:
df.head()

In [ ]:
while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_testing2', value=dict_stock)
    sleep(1)

In [ ]:
producer.flush()

### Consumer

In [ ]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps, loads, dump
import json
from s3fs import S3FileSystem

In [ ]:
consumer = KafkaConsumer(
    'demo_testing2',
    bootstrap_servers=['13.49.238.129:9092'],
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

In [ ]:
s3 = S3FileSystem()

In [ ]:
for count, i in enumerate(consumer):
    with s3.open("s3://{YOUR S3 BUCKET}/stock_market_{}.json".format(count), 'w') as file:
        json.dump(i.value, file) 